In [1]:
from stable_baselines3.ppo import PPO
from stable_baselines3.common.policies import ActorCriticCnnPolicy
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.vec_env.subproc_vec_env import SubprocVecEnv
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from wrapper import DoomEnvironment
from callback import CallBack

In [2]:
map_name = "deathmatch"
#DoomEnvironment.check_environment()
#DoomEnvironment.show_difference(map_name)

In [3]:
num_cpu = 6
render = False
one = num_cpu == 1

In [4]:
if one:
    env = DoomEnvironment(map_name=map_name,render=True)
else:
    env = [lambda : DoomEnvironment(map_name=map_name,render=render) for _ in range(num_cpu-1)]
    env.append(lambda : DoomEnvironment(map_name=map_name,render=True))
    env = SubprocVecEnv(env)

In [5]:
callback = CallBack(check_freq=10_000,save_path=f"train1/{map_name}")

In [6]:
#callback.n_calls = 60_001

In [7]:
model = PPO('CnnPolicy', env, verbose=1, learning_rate=0.01, n_steps=1_024, device='cuda')

Using cuda device
Wrapping the env in a VecTransposeImage.


In [8]:
#model.load(f"./train/{map_name}/final.zip")
if False:
    model = PPO.load(f"./train/{map_name}/final.zip")
    model.learning_rate = 0.1
    model.verbose = 1
    model.n_steps=1_024
    model.device='cuda'
    model.set_env(env)

In [9]:

model.learn(total_timesteps=150_000, callback=callback)

-----------------------------
| time/              |      |
|    fps             | 180  |
|    iterations      | 1    |
|    time_elapsed    | 33   |
|    total_timesteps | 6144 |
-----------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 156       |
|    iterations           | 2         |
|    time_elapsed         | 78        |
|    total_timesteps      | 12288     |
| train/                  |           |
|    approx_kl            | 19.014175 |
|    clip_fraction        | 0.847     |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.07     |
|    explained_variance   | 0.000384  |
|    learning_rate        | 0.01      |
|    loss                 | 16.8      |
|    n_updates            | 10        |
|    policy_gradient_loss | 0.359     |
|    value_loss           | 148       |
---------------------------------------
--------------------------------------
| time/                   |    

In [ ]:
model.save(f"./train1/{map_name}/final")

In [ ]:
env.close()